In [1]:
from __future__ import print_function
import argparse

import easydict

import cv2
import os

import shutil


In [2]:
def detectAndDisplay(img, font, parameter_config):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces, faceRejectLevels, faceLevelWeights= face_cascade.detectMultiScale3(gray, 1.3, 5, outputRejectLevels=True)
    f=0
    g=0
    z=0
    status = {
           'smile': False,
           'leftEyeOpen': False,
           'rightEyeOpen': False
       }
   
    for(x,y,w,h) in faces:
       
        if (round(faceLevelWeights[f][0],3)) <= parameter_config['faceSensitivity']: continue
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        cv2.putText(img, str(round(faceLevelWeights[f][0],3)), (x,y),font, 1, (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        # Smiles
        smiles, rejectLevels, levelWeights = smile_cascade.detectMultiScale3(roi_gray, outputRejectLevels=True)
        i=0
        for(ex,ey,ew,eh) in smiles:
            if(round(levelWeights[i][0],3)>=parameter_config['smileSensitivity']):
                status['smile'] = True

                cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,255,0), 2)
                cv2.putText(roi_color,str(round(levelWeights[i][0],3)),(ex,ey), font,1,(0,255,0),2)
            i+=1
        f+=1
        
        # Eyes
        left_eye, rejectLevels, eyelevelWeights = left_eye_cascade.detectMultiScale3(roi_gray, outputRejectLevels=True)
        j=0
        
        for(ex,ey,ew,eh) in left_eye:
            if(round(eyelevelWeights[j][0],3)>=parameter_config['leftEyeSensitivity']):
                status['leftEyeOpen'] = True

                cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (255,255,255), 2)
                cv2.putText(roi_color,str(round(eyelevelWeights[j][0],3)),(ex,ey), font,1,(255,255,255),2)
            j+=1
        g+=1
        
        
        
        right_eye, rejectLevels, eyelevelWeights = right_eye_cascade.detectMultiScale3(roi_gray, outputRejectLevels=True)

        j=0
        for(ex,ey,ew,eh) in right_eye:
            if(round(eyelevelWeights[j][0],3)>=parameter_config['rightEyeSensitivity']):
                status['rightEyeOpen'] = True

                cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (220,125,223), 2)
                cv2.putText(roi_color,str(round(eyelevelWeights[j][0],3)),(ex,ey), font,1,(220,125,223),2)
            j+=1
        z+=1      
    cv2.imshow('img',img)
    return img, status

In [3]:
parser = argparse.ArgumentParser()

args = parser.parse_args(args=[])
args = easydict.EasyDict({
    "face_cascade": 'xml/haarcascade_frontalface_alt.xml',
    "right_eye_cascade": 'xml/haarcascade_righteye_2splits.xml',
    "left_eye_cascade": 'xml/haarcascade_lefteye_2splits.xml',
    "smile_cascade": 'xml/haarcascade_smile.xml',
    'camera': 0,
    'maxPhotoes': 5,  # MAX NUMBER OF PHOTOES TO SAVE before shutting down the application
    'captureID': 1,  # this number goes at the end of name of the saved image, e.g capture1.png
    
    "parameter_config": {
        'leftEyeSensitivity': 3.5,  # at least how much sensible is left eye capture
        'rightEyeSensitivity': 3.5,  # at least how much sensible is right eye capture
        'faceSensitivity': 1,  #face sensitivity
        'smileSensitivity': 3.5,  # at least how much sensible is smile capture
    }
})


In [4]:
face_cascade_name = args.face_cascade
left_eye_cascade_name = args.left_eye_cascade
right_eye_cascade_name = args.right_eye_cascade
smile_cascade_name = args.smile_cascade
camera_device = args.camera
parameter_config = args.parameter_config    
captureID = args.captureID    
maxPhotoes = args.maxPhotoes    


face_cascade = cv2.CascadeClassifier()
left_eye_cascade = cv2.CascadeClassifier()
right_eye_cascade = cv2.CascadeClassifier()
smile_cascade = cv2.CascadeClassifier()

In [5]:
# Some validations


if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not left_eye_cascade.load(cv2.samples.findFile(left_eye_cascade_name)):
    print('--(!)Error loading left eye cascade')
    exit(0) 
    
if not right_eye_cascade.load(cv2.samples.findFile(right_eye_cascade_name)):
    print('--(!)Error loading right eye cascade')
    exit(0)
if not smile_cascade.load(cv2.samples.findFile(smile_cascade_name)):
    print('--(!)Error loading smile cascade')
    exit(0)    
    
cap = cv2.VideoCapture(camera_device)
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0) 


In [6]:
# Photoes Foldier creation

if os.path.isdir('Photoes'):
    shutil.rmtree('Photoes')
    os.mkdir('Photoes')
else :
    os.mkdir('Photoes')
    
# Note: if you find error in this cell run this cell the second time and error will vanish    

In [7]:
# Begin the display     
num_photos = 1
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
        
    font=cv2.FONT_HERSHEY_COMPLEX_SMALL
    img, status = detectAndDisplay(frame, font, parameter_config)
    if cv2.waitKey(10) == 27:
        break

   
    
    if (status['smile'] & status['leftEyeOpen']  & status['rightEyeOpen'] ):    
            cv2.imwrite('Photoes/capture' + str(captureID) + '.png', img)
    
            captureID = captureID + 1
            num_photos = num_photos + 1
        
    if num_photos > maxPhotoes:
            break   
    
cap.release()
cv2.destroyAllWindows()    

In [8]:
# just in case it did not fully shut down
cap.release()
cv2.destroyAllWindows()  